In [ ]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from IPython.display import Image
import numpy as np
from PIL import Image

#creates a data generator object that transform the images
datagen = ImageDataGenerator(rotation_range =  40,
                            width_shift_range = 0.2,
                            height_shift_range = 0.2,
                             shear_range = 0.2,
                             zoom_range = 0.2,
                             horizontal_flip = True,
                             vertical_flip = True,
                             fill_mode = 'nearest')

#pick an image to transform 

test_img = Image.open("imgkp.jpg")
img = image.img_to_array(test_img)          #converting to array
img = img / 255                             #normalise pixel value between 1 and 0
img = img.reshape((1,)+img.shape)

i=0

for batch in datagen.flow(img,save_prefix ='test',save_format = 'jpeg'):
    plt.figure(i)
    plot = plt.imshow(image.img_to_array(batch[0]))
    i+=1
    if i > 10:
        break;
plt.show()        

In [4]:
import face_recognition
import os
import cv2


KNOWN_FACES_DIR = 'known_faces'
UNKNOWN_FACES_DIR = 'unknown_faces'
TOLERANCE = 0.5
FRAME_THICKNESS = 3
FONT_THICKNESS = 2
MODEL = 'hog' #hog or cnn can be used 


# Returns (R, G, B) from name
def name_to_color(name):
    # Take 3 first letters, tolower()
    # lowercased character ord() value rage is 97 to 122, substract 97, multiply by 8
    color = [(ord(c.lower())-97)*8 for c in name[:3]]
    return color


print('Loading known faces...')
print('successfully loaded')
known_faces = []
known_names = []

#iterate over folders in known_faces_dir
for name in os.listdir(KNOWN_FACES_DIR):

 # load every face of known person
    for filename in os.listdir(f'{KNOWN_FACES_DIR}/{name}'):

        # Load an image
        image = face_recognition.load_image_file(f'{KNOWN_FACES_DIR}/{name}/{filename}')

        encoding = face_recognition.face_encodings(image)[0]

        known_faces.append(encoding)
        known_names.append(name)


print('Processing unknown faces...')

for filename in os.listdir(UNKNOWN_FACES_DIR):
   
    # Load image
    print(f'Filename {filename}', end='')
    image = face_recognition.load_image_file(f'{UNKNOWN_FACES_DIR}/{filename}')

    locations = face_recognition.face_locations(image, model=MODEL)

    #passing locations here makes process fast
    encodings = face_recognition.face_encodings(image, locations)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

   
    print(f', found {len(encodings)} faces')
    
    for face_encoding, face_location in zip(encodings, locations):

        results = face_recognition.compare_faces(known_faces, face_encoding, TOLERANCE)

        match = None
        if True in results:  
            match = known_names[results.index(True)]
          
            top_left = (face_location[3], face_location[0])
            bottom_right = (face_location[1], face_location[2])

            color = name_to_color(match)

            # drawing frame
            cv2.rectangle(image, top_left, bottom_right, color, FRAME_THICKNESS)

            top_left = (face_location[3], face_location[2])
            bottom_right = (face_location[1], face_location[2] + 22)

            cv2.rectangle(image, top_left, bottom_right, color, cv2.FILLED)

            # Wite a name
            cv2.putText(image, match, (face_location[3] + 10, face_location[2] + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), FONT_THICKNESS)

    # Show image
    cv2.imshow(filename,image)
    cv2.waitKey(0)
    cv2.destroyWindow(filename)

Loading known faces...
successfully loaded
Processing unknown faces...
Filename 1373491c-dfda-4c83-bdb8-ba80c574f148.JPG, found 6 faces
Filename 328f027c-bfd6-4ba7-8dcf-e45e7103add9.JPG, found 4 faces
Filename 55708ebc-1bb2-4e4b-baa0-bde104121177.JPG, found 5 faces
Filename 6f523634-8e24-4a2a-818a-46a773c132c3.JPG, found 1 faces
Filename 6fa8f6ab-e830-4e02-8881-ea00039a23a3.JPG, found 4 faces
Filename 70f44b0a-fd58-4c47-88e1-a4ed8627eeaa.JPG, found 7 faces
Filename IMG-0519.jpg, found 4 faces
Filename IMG-0534.jpg, found 4 faces
Filename IMG-0536.jpg, found 3 faces
Filename IMG-2763.JPG, found 1 faces
Filename IMG-6866.JPG, found 1 faces
Filename IMG-6873.jpg, found 1 faces
Filename IMG-6886.jpg, found 1 faces
Filename IMG-7181.JPG, found 2 faces
Filename IMG-7238.JPG, found 4 faces
